# 各DFの意味
## <eye_df>
実験課題開始から終了までの約10分の視線データ

### ・gx,gy
視線のディスプレイ上重心座標
### ・epoch_sec,h:mm:ss
各視線データ計測時のUNIX時間及びリアルタイム
### ・validity_sum
各視線データの左右の目の有効性の和
(0:どちらもNaN,1:どちらかNaN,2:どちらもデータが取れた)
### ・trial
各視線データがどの画像提示中に記録されたものかを表す
(-1:画像提示以外の時間)
## <trial_df>
実験課題実施中の行動データ

### ・Images
提示画像名
### ・trial
各画像のナンバリング
### ・image_epoch,image_str
画像提示開始時刻のUNIX時間及びリアルタイム
### ・hyouka_epoch,hyouka_str
画像評価開始時刻のUNIX時間及びリアルタイム
## <sampling_df>
各画像のサンプリングレート

### ・trial  
各画像のナンバリング
### ・samples
各画像提示中の視線データ数
### ・start_sec
画像提示開始時刻のUNIX時間
### ・end_sec
画像評価開始時刻
### ・duration_sec  
画像提示秒
### ・sampling_rate_Hz
各画像のサンプリングレート

In [20]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime,timezone
import os



# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [21]:
# # yyyy/mm/dd h:mm:ss.000
# 被験者IDを指定（例：id001）
participant_id = "id018"
exp_num = "002"
folder = "exported_csv"

# ファイル読み込み
psychopy_path = f"PsychoData/{participant_id}/{exp_num}.csv"
eye_path = f"tobiiData/aha-{participant_id}-{exp_num}.csv"

eye_df = pd.read_csv(eye_path)
psychopy_df = pd.read_csv(psychopy_path)


In [22]:
# 座標整形：gx, gy（左右平均）を作成
def parse_coord(coord_str):
    try:
        x, y = ast.literal_eval(coord_str)
        return float(x), float(y)
    except:
        return np.nan, np.nan

eye_df[["lx", "ly"]] = eye_df["left_gaze_point_on_display_area"].apply(parse_coord).apply(pd.Series)
eye_df[["rx", "ry"]] = eye_df["right_gaze_point_on_display_area"].apply(parse_coord).apply(pd.Series)
eye_df["gx"] = eye_df[["lx", "rx"]].mean(axis=1)
eye_df["gy"] = eye_df[["ly", "ry"]].mean(axis=1)

# validityの追加
eye_df["validity_sum"] = eye_df["left_gaze_point_validity"] + eye_df["right_gaze_point_validity"]


In [23]:
# Tobii出力データのエポック時間とhh:mm:ssの追加

# 1. datetime型に変換
eye_df["realtime"] = pd.to_datetime(eye_df["realtime"], format="%Y/%m/%d %H:%M:%S.%f")
eye_df["realtime"] = eye_df["realtime"].dt.tz_localize("Asia/Tokyo")

# 2. エポック秒（float型 or 整数型）
eye_df["epoch_sec"] = eye_df["realtime"].astype("int64") / 10**9  # 小数点付き


# 3. hh:mm:ss.000 形式の列を追加
eye_df["hhmmss"] = eye_df["realtime"].dt.strftime("%H:%M:%S.%f").str.slice(0, 12)


KeyboardInterrupt: 

In [ ]:
# Imagesがある行だけに加えて、.started列が両方とも有効な行だけ使う
valid_mask = (
    psychopy_df["Images"].notna() &
    psychopy_df["image.started"].notna() &
    psychopy_df["hyouka.started"].notna()
)

# 抽出
image_rows = psychopy_df[valid_mask].reset_index(drop=True)


In [ ]:
# PsychoPyログから開始時刻文字列を取得
exp_start_str = psychopy_df["expStart"].dropna().iloc[0]

# 文字列を datetime型に変換（タイムゾーン付き）
exp_start_dt = datetime.strptime(exp_start_str, "%Y-%m-%d %Hh%M.%S.%f %z")

# UTC基準のUNIX時間（float秒）に変換
exp_start_epoch = exp_start_dt.astimezone(timezone.utc).timestamp()


In [ ]:
# trialdfの作成
trial_df = image_rows[["Images"]].copy()
trial_df["trial"] = trial_df.index

for col in ["image.started", "hyouka.started"]:
    base = col.split(".")[0]
    relative_sec = image_rows[col].astype(float)

    trial_df[f"{base}_epoch"] = exp_start_epoch + relative_sec
    trial_df[f"{base}_str"] = pd.to_datetime(
        trial_df[f"{base}_epoch"], unit="s", utc=True
    ).dt.tz_convert("Asia/Tokyo").dt.strftime("%H:%M:%S.%f").str[:12]


In [ ]:
# 初期化：視線が属さない行は -1 にしておく
eye_df["trial"] = -1

# trial_df の各行（試行）についてループ
for _, row in trial_df.iterrows():
    trial_num = row["trial"]
    start_epoch = row["image_epoch"]
    end_epoch = row["hyouka_epoch"]

    # 視線データの中で、この試行の提示～評価前に含まれるものを選ぶ
    mask = (eye_df["epoch_sec"] >= start_epoch) & (eye_df["epoch_sec"] < end_epoch)

    # 条件に合う行に trial 番号を付ける
    eye_df.loc[mask, "trial"] = trial_num

# 抽出したい主要列（trial列を後で付けるなら省略可）
columns = [
    "gx", "gy", "epoch_sec", "hhmmss","validity_sum","trial"
]


# 再構成
eye_df = eye_df[columns].copy()



In [ ]:
# 1. pose.started列の最初の数値（開始相対秒）
pose_start_sec = float(psychopy_df["pose.started"].dropna().iloc[0])

# 2. pose.stopped列の最後の数値（終了相対秒）
pose_stop_sec = float(psychopy_df["pose.stopped"].dropna().iloc[-1])

# 3. UNIX時間に変換
pose_start_epoch = exp_start_epoch + pose_start_sec
pose_stop_epoch = exp_start_epoch + pose_stop_sec

# 4. 実験課題実施中の視線データを抽出
eye_exp = eye_df[(eye_df["epoch_sec"] >= pose_start_epoch) & (eye_df["epoch_sec"] <= pose_stop_epoch)]

# 5. サンプル数・時間・Hz
samples_exp = len(eye_exp)
duration_exp = pose_stop_epoch - pose_start_epoch
hz_exp = samples_exp / duration_exp if duration_exp > 0 else float("nan")


In [ ]:
# trial が有効な視線だけ使用
eye_trial = eye_df[eye_df["trial"] >= 0]

# 各試行ごとに画像提示中の視線サンプル数とHzを算出
sampling_df = eye_trial.groupby("trial").agg(
    samples=("epoch_sec", "count"),
    start_sec=("epoch_sec", "min"),
    end_sec=("epoch_sec", "max")
).reset_index()
sampling_df["duration_sec"] = sampling_df["end_sec"] - sampling_df["start_sec"]
sampling_df["sampling_rate_Hz"] = sampling_df["samples"] / sampling_df["duration_sec"]


In [ ]:

# 実験課題実施中の視線データを抽出
eye_df_trimmed = eye_df[
    (eye_df["epoch_sec"] >= pose_start_epoch) &
    (eye_df["epoch_sec"] <= pose_stop_epoch)
].copy()

# 実際に得られた視線データの期間（有効な最初と最後のタイムスタンプを使う）
start = eye_df_trimmed["epoch_sec"].min()
end = eye_df_trimmed["epoch_sec"].max()
duration_actual = end - start

# サンプル数
samples = len(eye_df_trimmed)

# 正味のサンプリング周波数（Hz）
hz_actual = samples / duration_actual if duration_actual > 0 else float("nan")

# 表示
print(f"[実験課題中 実測ベース]")
print(f"・記録された視線サンプル数: {samples} 点")
print(f"・記録実時間（視線の実データ間）: {duration_actual:.2f} 秒")
print(f"・実サンプリング周波数: {hz_actual:.2f} Hz")

print()

[実験課題中 実測ベース]
・記録された視線サンプル数: 51210 点
・記録実時間（視線の実データ間）: 600.00 秒
・実サンプリング周波数: 85.35 Hz



In [ ]:
# サマリー情報を1行のデータフレームとして作成
exp_summary_df = pd.DataFrame([{
    "participant": participant_id,
    "exp_num": exp_num,
    "pose_start_epoch": pose_start_epoch,
    "pose_stop_epoch": pose_stop_epoch,
    "duration_nominal": pose_stop_epoch - pose_start_epoch,  # 理論上の時間
    "samples": samples,
    "duration_actual": duration_actual,
    "sampling_rate_Hz": hz_actual
}])

In [ ]:
# debug用の出力
# print("===========================")
# print(eye_df.head(10))
# print("===========================")
# print(trial_df.head(10))
# print("===========================")    
# print(sampling_df.head(10))
# print("===========================")
# print(exp_summary_df.head(10))
# print("===========================")


In [ ]:
prefix = f"{participant_id}-{exp_num}"
print(f"これらのDFは{prefix}として保存します。")
# 1. 視線データの保存
eye_df.to_csv(f"{folder}/eye_df_{prefix}.csv", index=False, float_format="%.6f", encoding="utf-8-sig")

# 2. 試行データの保存
trial_df.to_csv(f"{folder}/trial_df_{prefix}.csv", index=False, float_format="%.6f", encoding="utf-8-sig")

# 3. サンプリング統計の保存
sampling_df.to_csv(f"{folder}/sampling_df_{prefix}.csv", index=False, float_format="%.6f", encoding="utf-8-sig")

これらのDFはid018-002として保存します。


In [ ]:
# ファイル名
summary_path = f"{folder}/exp_summary_all.csv"

# ヘッダー付きで新規保存 or ヘッダー無しで追記
if not os.path.exists(summary_path):
    exp_summary_df.to_csv(summary_path, index=False, float_format="%.6f", encoding="utf-8-sig")
else:
    exp_summary_df.to_csv(summary_path, mode='a', header=False, index=False, float_format="%.6f", encoding="utf-8-sig")
